In [1]:

#loading required packages.
import numpy as np
import pandas as pd
import re
import math
import xml.etree.ElementTree as ET


In [2]:

#Function to open and read the STEP file.
def Step_listing(path):
    #Opening the STEP file.
    main_file = open(path)
    #Reading the STEP file.
    file = main_file.readlines()    
    #Declaring a list for storing data by lines.
    data = []
    #For loop for reading the file.
    for i in range(len(file)):
        #Condition for reading only dimesnional data and not meta data of the file.
        if file[i][0] == 'D' and file[i][1] == 'A':
            #Recording data starting line.
            data_start_index = i

    #Storing data in list by each line.
    for i in range(data_start_index, len(file) - 2):
        data.append(file[i])
    return data


#Function for pulling out advance face line numbers.
def closed_shell_reader(data):
    for i in range(len(data)):
        #Storing Closed_Shell info in a variable.
        if (data[i].count('CLOSED_SHELL') > 0):
            closed_shell = data[i]
    #Listing ut all line numbers present in closed shell line.
    all_nums = re.findall("\d+", closed_shell)
    #Finding all advanced face line numbers. i.e removing first position number since it represent closed shell's line.
    closed_shell_line = all_nums[0]
    advanced_faces_line = all_nums[1:]
    return closed_shell_line, advanced_faces_line

#Function for pulling out Face outer bound and surface line numbers.
def advance_face_reader(data, advanced_faces_line):
    all_nums = []
    #Reading each faces line number.
    for faces in advanced_faces_line:
        faces = int(faces)
        all_nums.append(re.findall("\d+", data[faces]))
    face_outer_bounds_lines = []
    surface_lines = []
    for nums in range (len(all_nums)):
        #Storing all face outer bound line numbers in a list.
        face_outer_bounds_lines.append(all_nums[nums][1])
        #Storing all surface information line numbers in a list.
        surface_lines.append(all_nums[nums][2])
    return face_outer_bounds_lines, surface_lines

def Step_reader(path):
    data = Step_listing(path)
    closed_shell_line, advanced_faces_line = closed_shell_reader(data)
    print("Closed Shell Line Number: ", closed_shell_line)
    print("Advanced Faces: ", advanced_faces_line, '\n', '\n')
    face_outer_bounds_lines, surface_lines = advance_face_reader(data, advanced_faces_line)
    return face_outer_bounds_lines, surface_lines, data

#Function for reading edge loop line numbers from given face outer bond line number.
def Edge_Loop_Finder(face_outer_bound, data):
    #Reading each face outer bound.
    Edge_loops = re.findall("\d+", data[face_outer_bound])
    Edge_loops = Edge_loops[1:]
    return Edge_loops

#Function for reading oriented edge line numbers from given Edge loop line number.
def Oriented_Edge_Finder(Edge_loop, data):
    #Reading each edge loop line.
    Oriented_Edges = re.findall("\d+", data[Edge_loop])
    Oriented_Edges = Oriented_Edges[1:]
    return Oriented_Edges

#Function for reading Edge curve line numbers from given Oriented edge line number.
def Edge_Curve_Finder(Oriented_Edge, data):
    #Reading each edge curve line.
    Edge_Curve = re.findall("\d+", data[Oriented_Edge])
    Edge_Curve = Edge_Curve[1:]
    return Edge_Curve

#Function for reading Cartesian Point line numbers from given Edge curve line number.
def Cartesian_Point_Finder(Edge_Curve, data):
    #Reading each edge curve line.
    Edge_Curve_content = re.findall("\d+", data[Edge_Curve])
    Virtex_Points = []
    Circular = []
    Lines = []
    #Detecting form of data in given line.
    for text in Edge_Curve_content:
        if "VERTEX_POINT" in data[int(text)]:
            Virtex_Points.append(text)
        elif "CIRCLE" in data[int(text)]:
            Circular.append(text)
        elif "LINE" in data[int(text)]:
            Lines.append(text)
    Catretian_Point_lines = []
    #Reading each cartesian point line number respective to the virtex points.
    for Virtex_Point in Virtex_Points:
        Virtex_Point = int(Virtex_Point)
        #Reading each edge curve line.
        Catretian_Point_line = re.findall("\d+", data[Virtex_Point])
        Catretian_Point_line = int(Catretian_Point_line[1])
        Catretian_Point_lines.append(Catretian_Point_line)
    return Catretian_Point_lines, Circular, Lines

#Function for reading Cartesian Point line numbers from given Edge curve line number.
def actual_cartesian_points(Catretian_Point_lines, data):
    Catretian_Points = []
    for Catretian_Point_line in Catretian_Point_lines:
        #Reading each cartesian point line.
        x = (re.findall("\d+[.]+\d+[\w]+.+\d+", data[Catretian_Point_line]))
        Catretian_Points.append(x[0].split(", "))

    #Converting the string numbers into floating points numbers.
    New_Catretian_Points = []
    for points in Catretian_Points:
        refined_point = []
        for point in points:
            #Condition for converting to decimal point.
            if 'E' in point:
                left_side = []
                index = 0
                while index < len(point):
                    if point[index] == "E":
                        break
                    index += 1
                left_side = point[: index]
                right_side = point[(index + 2):]
                if point[index + 1] == "-":
                    din = "1"
                    for i in range(int(right_side)):
                        din = din + "0"
                    act_point = float(left_side)/int(din)
                else:
                    num = "1"
                    for i in range(int(right_side)):
                        num = num + "0"
                    act_point = float(left_side)*int(num)
            else:
                act_point = float(point)
            refined_point.append(act_point)
        New_Catretian_Points.append(refined_point)
    return New_Catretian_Points

#Function for reading circular radius from given circular line number.
def radius_finder(Circular, data):
    #Reading each radius.
    radius = re.findall("\d+[.]+\d+", data[Circular])
    return radius


#Function for calculating distance between given two points.
def distance_finder(points):
    x_axis = []
    y_axis = []
    z_axis = []
    result_dist = []
    for point in points:
        x_axis.append(point[0])
        y_axis.append(point[1])
        z_axis.append(point[2])
    #Calculating distance between points at each axis.
    x1 = x_axis[0]
    x2 = x_axis[1]
    y1 = y_axis[0]
    y2 = y_axis[1]
    z1 = z_axis[0]
    z2 = z_axis[1]
    
    result_dist = math.sqrt(pow((x2 - x1), 2) + pow((y2 - y1), 2) + pow((z2 - z1), 2))    
    return result_dist


#Overall Function for extracting both dimensional and radial dimensions from given STEP file.
def complete_STEP_extractor(file_path):
    #Reading face outer bounds.
    face_outer_bounds_lines, surface_lines, data = Step_reader(file_path)

    #Creating the file structure for each component.
    Component_1 = ET.Element('Component_1')

    #Edge loop for each face outer bond.
    Edge_loops = []
    radiuses = []
    distances = []
    cp = []
    ed = []
    a_cp = []
    curvy_find = []
    print("Face Outer Bounds: ", face_outer_bounds_lines, '\n')
    for face_outer_bound in face_outer_bounds_lines:
        print("------------------------------------------------------------------------")
        print("Face Outer Bound: ", int(face_outer_bound))
        
        #Creating feature based on face number.
        #Feature = ET.SubElement(Component_1, str('Feature_') + str(face_outer_bound))
        Feature = ET.SubElement(Component_1, 'Feature')
        Feature.set('Feature_Number', str(face_outer_bound))
        
        print("----------------------", '\n')
        Edge_loop = Edge_Loop_Finder(int(face_outer_bound), data)
        print('\t', "Edge Loop: ", Edge_loop[0])
        Edge_loops.append(int(Edge_loop[0]))
        #print('\t', "Edge Loops: ", Edge_loops)
        for Edge_loop in Edge_loops:
            Oriented_Edges = Oriented_Edge_Finder(Edge_loop, data)
        print('\t', "Oriented Edges: ", Oriented_Edges, '\n')
        for Oriented_Edge in Oriented_Edges:
            print('\t', '\t', "Oriented Edge: ", Oriented_Edge)
            Edge_Curves = Edge_Curve_Finder(int(Oriented_Edge), data)
            print('\t', '\t', "Edge Curves: ", Edge_Curves, '\n')
            for Edge_Curve in Edge_Curves:
                print('\t', '\t', '\t', "Edge Curve: ", Edge_Curve)
                Catretian_Point_lines, Circular, Lines = Cartesian_Point_Finder(int(Edge_Curve), data)
                print('\t', '\t', '\t', '\t', "Catretian Point lines: ", Catretian_Point_lines)
                points = actual_cartesian_points(Catretian_Point_lines, data)
                cp.append(Catretian_Point_lines)
                ed.append(Edge_Curve)
                a_cp.append(points)
                #print("Cartesian Points: ", points)
                #Condition for circular edge.
                if Circular != []:
                    for circle in Circular:
                        radius = radius_finder(int(circle), data)
                        radiuses.append(radius[0])
                        print('\t', '\t', '\t', '\t', '\t', "Radius: ", radius)
                        curvy_find.append(True)
                        Dimensions_Circular = ET.SubElement(Feature, 'Dimensions_Circular')
                        Dimensions_Circular.set('Radius', radius[0])

                else:
                    distance = distance_finder(points)
                    distances.append(distance)
                    curvy_find.append(False)
                    print('\t', '\t', '\t', '\t', '\t', "Length is: ", distance, '\n')
                    Dimensions_Linear = ET.SubElement(Feature, 'Dimensions_Linear')
                    Dimensions_Linear.set('Length', str(distance))
    
    return radiuses, distances, ed, cp, curvy_find, a_cp, Component_1


#Function for finding unique values from list.
def unique(raw_list):
    unique_list = []
    position = []
    for x in range(len(raw_list)):
        if raw_list[x] not in unique_list:
            unique_list.append(raw_list[x])
            position.append(x)
    return unique_list, position




In [4]:

#Part names to be evaluvated.
part_name = 'DDFlange'
#Path of file.
file_path = ('C:/' + part_name + '.STEP')


radiuses, distances, ed, cp, curvy_find, a_cp, Component_1 = complete_STEP_extractor(file_path)

#Creating new XML file with the dimensions.
component_data = ET.tostring(Component_1)
new_file = open("DDFlange.xml", "wb")
new_file.write(component_data)
new_file.close()




Closed Shell Line Number:  2397
Advanced Faces:  ['1420', '1434', '1449', '1470', '1485', '1500', '1515', '1582', '1597'] 
 



IndexError: list index out of range